# Tamil News Title Tokenizer

This notebook contains a CSV-based preprocessing pipeline for Tamil news title classification.
Cells are laid out as: imports, tokenizer class, resource creation, initialization, and processing.

In [79]:
import re
import pandas as pd
from typing import List
import os

In [80]:
class TamilNewsTokenizer:
    
    
    def __init__(self, stopwords_file: str, suffixes_file: str):
       
        self.stopwords = set()
        self.suffixes = {}
        
        # Load stopwords
        self._load_stopwords(stopwords_file)
        
        # Load suffixes
        self._load_suffixes(suffixes_file)
        
        # for greedy matching
        self.sorted_suffixes = sorted(self.suffixes.keys(), key=len, reverse=True)
        
        print(f"Tokenizer initialized")
        print(f"  Stopwords: {len(self.stopwords)}")
        print(f"  Suffixes: {len(self.suffixes)}")
    
    
    def _load_stopwords(self, filepath: str):
        """Load stopwords from file (one per line)."""
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Stopwords file not found: {filepath}")
        
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                self.stopwords = {line.strip() for line in f if line.strip() and not line.startswith('#')}
            print(f"Loaded {len(self.stopwords)} stopwords from {filepath}")
        except Exception as e:
            raise Exception(f"Error loading stopwords: {e}")
    
    
    def _load_suffixes(self, filepath: str):
        """Load suffixes from CSV file (format: suffix)."""
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Suffixes file not found: {filepath}")
        
        try:
            df = pd.read_csv(filepath, encoding='utf-8')            
            if 'suffix' not in df.columns or 'meaning' not in df.columns:
                raise ValueError("Suffixes CSV must have 'suffix' and 'meaning' columns")
            
            for _, row in df.iterrows():
                suffix = str(row['suffix']).strip()
                meaning = str(row['meaning']).strip()
                
                if suffix and meaning and not suffix.startswith('#'):
                    self.suffixes[suffix] = meaning
            
            print(f"Loaded {len(self.suffixes)} suffixes from {filepath}")
        except Exception as e:
            raise Exception(f"Error loading suffixes: {e}")
    
    
    def clean_text(self, text: str) -> str:
        """
        Clean text for news title classification.
        - Remove digits and English letters
        - Remove punctuation except Tamil punctuation
        - Normalize whitespace
        """
        if pd.isna(text):
            return ""
        
        text = str(text)
        
        text = re.sub(r'[a-zA-Z]+', '', text)
        
        text = re.sub(r'[0-9௦-௯]+', '', text)
        
        text = re.sub(r'[!\"#$%&\'()*+,\-./:;<=>?@\[\\\\\]^_`{|}~…–—]', ' ', text)
        
        text = re.sub(r'[₹$€£¥●○■□★☆♦♥♠♣]', '', text)
        
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    
    def word_tokenize(self, text: str) -> List[str]:
        """Tokenize text into words."""
        if not text:
            return []
        return [w.strip() for w in text.split() if w.strip()]
    
    
    def remove_stopwords(self, words: List[str]) -> List[str]:
        """Remove Tamil stopwords."""
        return [w for w in words if w not in self.stopwords]
    
    
    def split_morphemes(self, word: str) -> List[str]:
        """
        Split word into morphemes (root + suffixes).
        Returns flat list of morphemes for easy joining.
        """
        if len(word) <= 2:
            return [word]
        
        morphemes = []
        remaining = word
        
        # Extract suffixes iteratively
        while len(remaining) > 2:
            suffix_found = False
            
            for suffix in self.sorted_suffixes:
                if remaining.endswith(suffix):
                    root = remaining[:-len(suffix)]
                    if len(root) >= 2:
                        morphemes.append(suffix)
                        remaining = root
                        suffix_found = True
                        break
            
            if not suffix_found:
                break
        
        morphemes.insert(0, remaining)
        
        return morphemes
    
    def remove_suffixes(self, words: List[str]) -> List[str]:
        """Remove Tamil suffixes."""
        return [w for w in words if w not in self.suffixes]


    def process_title(self, title: str, remove_stops: bool = True, remove_suffixes: bool = False,
                      split_morph: bool = True) -> str:
        # Clean
        cleaned = self.clean_text(title)
        
        # Tokenize
        words = self.word_tokenize(cleaned)
        
        # Remove stopwords if requested
        if remove_stops:
            words = self.remove_stopwords(words)
        
        # Split morphemes if requested
        if split_morph:
            all_morphemes = []
            for word in words:
                morphemes = self.split_morphemes(word)
                all_morphemes.extend(morphemes)
            words = all_morphemes
        else:
            # Remove suffixes
            if remove_suffixes:
                words = self.remove_suffixes(words)
            return ' '.join(words)

        if remove_suffixes:
            words = self.remove_suffixes(words)

        return ' '.join(words)
    
    
    def process_csv(self, input_csv: str, output_csv: str, 
                    title_column: str = 'title',
                    label_column: str = None,
                    remove_stopwords: bool = True,
                    remove_suffixes: bool = False,
                    split_morphemes: bool = True):

        print("\n" + "="*60)
        print("Tamil News Title Preprocessing")
        print("="*60)
        
        # Read CSV
        print(f"\n[1] Reading CSV: {input_csv}")
        df = pd.read_csv(input_csv, encoding='utf-8', sep =',')
        print(f"  ✓ Loaded {len(df)} rows")
        print(f"  Columns: {list(df.columns)}")
        
        # Check if title column exists
        if title_column not in df.columns:
            print(f"  ✗ Error: Column '{title_column}' not found!")
            print(f"  Available columns: {list(df.columns)}")
            return
        
        # Process titles
        print(f"\n[2] Processing titles...")
        print(f"  Remove stopwords: {remove_stopwords}")
        print(f"  Remove suffixes: {remove_suffixes}")
        print(f"  Split morphemes: {split_morphemes}")
        
        # Create new columns
        df['cleaned_title'] = df[title_column].apply(
            lambda x: self.clean_text(x)
        )
        
        df['tokenized_title'] = df['cleaned_title'].apply(
            lambda x: ' '.join(self.word_tokenize(x))
        )
        
        df['processed_title'] = df[title_column].apply(
            lambda x: self.process_title(x, remove_stopwords, remove_suffixes, split_morphemes)
        )
        
        # Calculate statistics
        print(f"\n[3] Statistics:")
        original_words = df[title_column].apply(
            lambda x: len(self.word_tokenize(self.clean_text(x)))
        ).sum()
        
        processed_tokens = df['processed_title'].apply(
            lambda x: len(x.split()) if x else 0
        ).sum()
        
        print(f"  Original word count: {original_words}")
        print(f"  Processed token count: {processed_tokens}")
        
        avg_original = df[title_column].apply(
            lambda x: len(self.word_tokenize(self.clean_text(x)))
        ).mean()
        
        avg_processed = df['processed_title'].apply(
            lambda x: len(x.split()) if x else 0
        ).mean()
        
        print(f"  Avg words per title (original): {avg_original:.2f}")
        print(f"  Avg tokens per title (processed): {avg_processed:.2f}")
        
        empty_count = (df['processed_title'] == '').sum()
        if empty_count > 0:
            print(f"  ⚠ Warning: {empty_count} titles became empty after processing")
            df = df[df['processed_title'] != '']
        
        print(f"\n[4] Saving to: {output_csv}")

        output_cols = ['processed_title']
        if label_column and label_column in df.columns:
            output_cols.insert(0, label_column)
        
        output_cols_full = output_cols + ['cleaned_title', 'tokenized_title', title_column]
        
        df[output_cols_full].to_csv(output_csv, index=False, encoding='utf-8')
        
        print(f"  ✓ Saved {len(df)} rows")
        print(f"  Columns: {output_cols_full}")
        
        print("\n" + "="*60)
        print("✓ Processing Complete!")
        print("="*60)
        
        return df

In [ ]:
# def create_resource_files():
#     """
#     Create sample resource files (stopwords and suffixes).
#     Run this cell only once to create template files.
#     """
#     print("Creating resource files...\n")
    
#     # Create resources directory
#     os.makedirs('resources', exist_ok=True)
    
#     # Create stopwords file
#     stopwords = """# Tamil Stopwords for News Classification
# # Lines starting with # are comments and will be ignored

# # Pronouns and demonstratives
# ஆக
# ஆகவே
# ஆதலால்
# ஆனால்
# என்
# என்ற
# என்று
# இது
# இதை
# இந்த
# இவை
# இவர்
# இல்லை
# உள்ள
# உள்ளது
# உள்ளன
# என
# எனவே
# எனில்
# ஒரு
# ஒன்று
# கூட
# தான்
# நான்
# நாம்
# நீ
# நீங்கள்
# பல
# பற்றி
# மற்றும்
# முதல்
# வேண்டும்
# அவர்
# அவன்
# அவள்
# அது
# அந்த
# போல
# போன்ற
# மட்டும்
# மிக
# வரை
# ஆம்
# ஏன்
# எப்படி
# எங்கே
# எப்போது
# எது
# எவர்
# சில
# பின்
# முன்
# மேல்
# கீழ்
# உடன்

# # Additional common words
# என்பது
# என்பதை
# என்பதால்
# இருந்து
# இருக்கும்
# இருக்கிறது
# செய்து
# செய்யும்
# செய்கிறது
# உடைய
# உள்ளிட்ட
# தொடர்பான
# சார்ந்த
# வந்த
# வரும்
# கொண்ட
# கொண்டு
# முடியும்
# முடிந்த
# பெற்ற
# பெறும்

# # Time-related stopwords (optional - remove if needed for news)
# இன்று
# நேற்று
# இனி
# பின்னர்
# தற்போது
# அண்மையில்
# சமீபத்தில்
# எதிர்வரும்
# மேலும்
# குறிப்பாக
# முக்கியமாக
# """
    
#     with open('resources/stopwords.txt', 'w', encoding='utf-8') as f:
#         f.write(stopwords)
#     print("✓ Created: resources/stopwords.txt")
    
#     # Create suffixes CSV
#     suffixes_data = """suffix,meaning
# ஐ,accusative
# ஆல்,instrumental
# ஆன்,genitive
# இல்,locative
# இலிருந்து,ablative
# இன்,genitive
# உக்கு,dative
# உடன்,sociative
# ஓடு,sociative
# கு,dative
# க்கு,dative
# அது,demonstrative
# எனில்,conditional
# கள்,plural
# களை,plural_acc
# களில்,plural_loc
# களுக்கு,plural_dat
# களால்,plural_inst
# களின்,plural_gen
# களோடு,plural_soc
# களுடன்,plural_soc
# கிறது,pres_neut
# கிறான்,pres_masc
# கிறாள்,pres_fem
# கிறார்,pres_hon
# கிறேன்,pres_1st
# கிறாய்,pres_2nd
# கிறீர்,pres_2nd_hon
# கிறோம்,pres_1st_pl
# கின்றது,pres_neut_formal
# கின்றான்,pres_masc_formal
# கின்றாள்,pres_fem_formal
# கின்றேன்,pres_1st_formal
# கின்றார்,pres_hon_formal
# கின்றனர்,pres_pl_formal
# ந்தது,past_neut
# ந்தான்,past_masc
# ந்தாள்,past_fem
# ந்தார்,past_hon
# ந்தேன்,past_1st
# ந்தாய்,past_2nd
# ந்தோம்,past_1st_pl
# ந்தனர்,past_pl
# த்தது,past_neut_alt
# த்தான்,past_masc_alt
# த்தாள்,past_fem_alt
# த்தார்,past_hon_alt
# த்தேன்,past_1st_alt
# ட்டது,past_neut_alt2
# ட்டான்,past_masc_alt2
# ட்டார்,past_hon_alt2
# வான்,fut_masc
# வாள்,fut_fem
# வார்,fut_hon
# வேன்,fut_1st
# வாய்,fut_2nd
# வோம்,fut_1st_pl
# வார்கள்,fut_pl
# ப்பான்,fut_masc_alt
# ப்பாள்,fut_fem_alt
# ப்பார்,fut_hon_alt
# ப்பேன்,fut_1st_alt
# கிற,pres_part
# கின்ற,pres_part_formal
# ந்த,past_part
# த்த,past_part_alt
# ட்ட,past_part_alt2
# உம்,part_conj
# ஆமல்,neg_part
# ஆது,neg_part_alt
# ஆன,adj
# ஆனது,rel_neut
# ஆனவர்,rel_person
# ஆனவை,rel_pl
# என்று,quotative
# இடம்,place
# படி,manner
# போல,like
# போது,time
# பின்,after
# முன்,before
# மட்டும்,only
# வரை,until
# ஆக,as
# ஆகவே,therefore
# ஆகிய,conjunction
# ஆகியோர்,conjunction_person
# ஆகியவை,conjunction_things
# ஆகியன,conjunction_neuter
# எனும்,called
# எனப்படும்,called_passive
# உடைய,possessive
# மிக்க,superlative
# உள்ளிட்ட,including
# சார்ந்த,related
# தொடர்பான,related_to
# பெற்ற,obtained
# கொண்ட,having
# செய்த,did
# செய்யும்,will_do
# செய்யப்பட்ட,done_passive
# செய்யப்படும்,being_done
# அளித்த,gave
# அளித்து,giving
# அளிக்கப்பட்ட,given_passive
# வழங்கிய,provided
# வழங்கும்,providing
# ஏற்பட்ட,occurred
# ஏற்படும்,will_occur
# நடந்த,happened
# நடக்கும்,will_happen
# நடைபெற்ற,took_place
# நடைபெறும்,will_take_place
# கூறிய,said
# கூறும்,saying
# தெரிவித்த,informed
# தெரிவிக்கும்,informing
# அறிவித்த,announced
# அறிவிக்கும்,announcing
# வந்த,came
# வரும்,coming
# சென்ற,went
# செல்லும்,going
# இருந்த,was
# இருக்கும்,will_be
# உள்ள,having_present
# தொடங்கிய,started
# தொடங்கும்,starting
# முடிந்த,finished
# முடியும்,will_finish
# கண்ட,saw
# காணும்,seeing
# ஆன,became
# ஆகும்,becoming
# """
    
#     with open('resources/suffixes.csv', 'w', encoding='utf-8') as f:
#         f.write(suffixes_data)
#     print("✓ Created: resources/suffixes.csv")
    
#     print("\n✓ Resource files created successfully!")
#     print("  You can now edit these files to add/remove stopwords and suffixes")


# # Run this function once to create template files
# create_resource_files()

Creating resource files...

✓ Created: resources/stopwords.txt
✓ Created: resources/suffixes.csv

✓ Resource files created successfully!
  You can now edit these files to add/remove stopwords and suffixes


In [81]:

tokenizer = TamilNewsTokenizer(
    stopwords_file='resources/stopwords.txt',
    suffixes_file='resources/suffixes.csv'
)

Loaded 127 stopwords from resources/stopwords.txt
Loaded 133 suffixes from resources/suffixes.csv
Tokenizer initialized
  Stopwords: 127
  Suffixes: 133


In [82]:
# Load your dataset
from pathlib import Path
path = Path('dataset/data.csv')
df = pd.read_csv(path, encoding="utf-8", sep=",")

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Dataset shape: (18453, 3)

Columns: ['title', 'extra', 'category']

First few rows:


,title,extra,category
0,"மேகதாது விவகாரம்: தமிழக, கர்நாடகா முதலமைச்சர்க...",https://www.puthiyathalaimurai.com/tamilnadu/t...,தமிழ்நாடு
1,பந்துவீச்சாளர்கள் ஐபிஎல் விளையாடலாமா? - எதிரெத...,https://www.puthiyathalaimurai.com/sports/ms-d...,விளையாட்டு
2,"கனமழை எச்சரிக்கை | நாளை பள்ளி, கல்லூரிகளுக்கு ...",https://www.puthiyathalaimurai.com/tamilnadu/t...,தமிழ்நாடு
3,தவெகவை ஆண்டவனாலும் காப்பாற்ற முடியாது - விஜய்க...,https://www.puthiyathalaimurai.com/tamilnadu/r...,தமிழ்நாடு
4,ஆழ்ந்த காற்றழுத்த தாழ்வுப்பகுதி எதிரொலி.. 8 மா...,https://www.puthiyathalaimurai.com/tamilnadu/d...,தமிழ்நாடு


In [83]:

processed_df = tokenizer.process_csv(
    input_csv='dataset/data.csv',
    output_csv='output/processed_data.csv',
    title_column='title',           
    label_column='category',        
    remove_stopwords=True,
    remove_suffixes=True,
    split_morphemes=True
)


Tamil News Title Preprocessing

[1] Reading CSV: dataset/data.csv
  ✓ Loaded 18453 rows
  Columns: ['title', 'extra', 'category']

[2] Processing titles...
  Remove stopwords: True
  Remove suffixes: True
  Split morphemes: True

[3] Statistics:
  Original word count: 159562
  Processed token count: 148735
  Avg words per title (original): 8.65
  Avg tokens per title (processed): 8.06
  ⚠ Warning: 5 titles became empty after processing

[4] Saving to: output/processed_data.csv
  ✓ Saved 18448 rows
  Columns: ['category', 'processed_title', 'cleaned_title', 'tokenized_title', 'title']

✓ Processing Complete!


In [84]:

print("\nProcessed Data Sample:")
processed_df[['title', 'cleaned_title', 'tokenized_title', 'processed_title']].head(10)


Processed Data Sample:


,title,cleaned_title,tokenized_title,processed_title
0,"மேகதாது விவகாரம்: தமிழக, கர்நாடகா முதலமைச்சர்க...",மேகதாது விவகாரம் தமிழக கர்நாடகா முதலமைச்சர்களு...,மேகதாது விவகாரம் தமிழக கர்நாடகா முதலமைச்சர்களு...,மேகதாது விவகாரம் தமிழக கர்நாடகா முதலமைச்சர் நி...
1,பந்துவீச்சாளர்கள் ஐபிஎல் விளையாடலாமா? - எதிரெத...,பந்துவீச்சாளர்கள் ஐபிஎல் விளையாடலாமா எதிரெதிர்...,பந்துவீச்சாளர்கள் ஐபிஎல் விளையாடலாமா எதிரெதிர்...,பந்துவீச்சாளர் ஐபிஎல் விளையாடலாமா எதிரெதிர் கர...
2,"கனமழை எச்சரிக்கை | நாளை பள்ளி, கல்லூரிகளுக்கு ...",கனமழை எச்சரிக்கை நாளை பள்ளி கல்லூரிகளுக்கு விட...,கனமழை எச்சரிக்கை நாளை பள்ளி கல்லூரிகளுக்கு விட...,கனமழை எச்சரிக்கை நாளை பள்ளி கல்லூரி விடுமுறை எ...
3,தவெகவை ஆண்டவனாலும் காப்பாற்ற முடியாது - விஜய்க...,தவெகவை ஆண்டவனாலும் காப்பாற்ற முடியாது விஜய்க்க...,தவெகவை ஆண்டவனாலும் காப்பாற்ற முடியாது விஜய்க்க...,தவெகவை ஆண்டவனாலும் காப்பாற்ற முடியாது விஜய் ஆர...
4,ஆழ்ந்த காற்றழுத்த தாழ்வுப்பகுதி எதிரொலி.. 8 மா...,ஆழ்ந்த காற்றழுத்த தாழ்வுப்பகுதி எதிரொலி மாவட்ட...,ஆழ்ந்த காற்றழுத்த தாழ்வுப்பகுதி எதிரொலி மாவட்ட...,ஆழ் காற்றழு தாழ்வுப்பகுதி எதிரொலி மாவட்டங் அலர்ட்
5,உருவானது புதிய காற்றழுத்த தாழ்வுப்பகுதி.. எங்க...,உருவானது புதிய காற்றழுத்த தாழ்வுப்பகுதி எங்கெல...,உருவானது புதிய காற்றழுத்த தாழ்வுப்பகுதி எங்கெல...,உருவானது புதிய காற்றழு தாழ்வுப்பகுதி எங்கெல்லா...
6,பாமகவை இழுக்க முயற்சிக்கும் NDA: கறார் காட்டும...,பாமகவை இழுக்க முயற்சிக்கும் கறார் காட்டும் ராம...,பாமகவை இழுக்க முயற்சிக்கும் கறார் காட்டும் ராம...,பாமகவை இழுக்க முயற்சிக்கும் கறார் காட்டும் ராம...
7,வங்கக்கடலில் காற்றழுத்த தாழ்வுப்பகுதி: தமிழகம்...,வங்கக்கடலில் காற்றழுத்த தாழ்வுப்பகுதி தமிழகம் ...,வங்கக்கடலில் காற்றழுத்த தாழ்வுப்பகுதி தமிழகம் ...,வங்கக்கடலில் காற்றழு தாழ்வுப்பகுதி தமிழகம் முழ...
8,அரபிக்கடலில் காற்றழுத்த தாழ்வு\r\nமண்டலம்.. நா...,அரபிக்கடலில் காற்றழுத்த தாழ்வு மண்டலம் நாளை மா...,அரபிக்கடலில் காற்றழுத்த தாழ்வு மண்டலம் நாளை மா...,அரபிக்கடலில் காற்றழு தாழ்வு மண்டலம் நாளை மாவட்...
9,அனிமேஷன் யுத்தம் | இணையத்தில் மல்லுக்கட்டும் த...,அனிமேஷன் யுத்தம் இணையத்தில் மல்லுக்கட்டும் திம...,அனிமேஷன் யுத்தம் இணையத்தில் மல்லுக்கட்டும் திம...,அனிமேஷன் யுத்தம் இணையத்தில் மல்லுக்கட்டும் திம...


In [ ]:
processed_df['tokenized_title'] = processed_df['cleaned_title'].apply(
    lambda x: tokenizer.word_tokenize(x) if isinstance(x, str) else []
)
for idx, tokens in processed_df['tokenized_title'].head(10).items():
    print(f"{idx} {tokens}")




0 ['மேகதாது', 'விவகாரம்', 'தமிழக', 'கர்நாடகா', 'முதலமைச்சர்களுக்கு', 'நிதின்', 'கட்கரி', 'கடிதம்']
1 ['பந்துவீச்சாளர்கள்', 'ஐபிஎல்', 'விளையாடலாமா', 'எதிரெதிர்', 'கருத்தில்', 'தோனி', 'கும்பளே']
2 ['கனமழை', 'எச்சரிக்கை', 'நாளை', 'பள்ளி', 'கல்லூரிகளுக்கு', 'விடுமுறை', 'எங்கெல்லாம்', 'தெரியுமா']
3 ['தவெகவை', 'ஆண்டவனாலும்', 'காப்பாற்ற', 'முடியாது', 'விஜய்க்கு', 'ஆர்பி', 'உதயகுமார்', 'அட்வைஸ்']
4 ['ஆழ்ந்த', 'காற்றழுத்த', 'தாழ்வுப்பகுதி', 'எதிரொலி', 'மாவட்டங்களுக்கு', 'அலர்ட்']
5 ['உருவானது', 'புதிய', 'காற்றழுத்த', 'தாழ்வுப்பகுதி', 'எங்கெல்லாம்', 'மழைக்கு', 'வாய்ப்பு']
6 ['பாமகவை', 'இழுக்க', 'முயற்சிக்கும்', 'கறார்', 'காட்டும்', 'ராமதாஸ்', 'அன்புமணி', 'வைக்கும்', 'கோரிக்கை', 'என்ன', 'நடக்கிறது']
7 ['வங்கக்கடலில்', 'காற்றழுத்த', 'தாழ்வுப்பகுதி', 'தமிழகம்', 'முழுவதும்', 'பரவலாக', 'மழை']
8 ['அரபிக்கடலில்', 'காற்றழுத்த', 'தாழ்வு', 'மண்டலம்', 'நாளை', 'மாவட்டங்களில்', 'கனமழைக்கு', 'வாய்ப்பு']
9 ['அனிமேஷன்', 'யுத்தம்', 'இணையத்தில்', 'மல்லுக்கட்டும்', 'திமுக', 'அதிமுக', 'ஐடிவிங்', 'என்ன', 'நடக்கிறது'

In [85]:

print("="*80)
print("COMPARISON: Original vs Processed")
print("="*80)

for idx in range(min(5, len(processed_df))):
    print(f"\n[Example {idx+1}]")
    print(f"Original:  {processed_df.iloc[idx]['title']}")
    print(f"Cleaned:   {processed_df.iloc[idx]['cleaned_title']}")
    print(f"Processed: {processed_df.iloc[idx]['processed_title']}")
    print("-"*80)

COMPARISON: Original vs Processed

[Example 1]
Original:  மேகதாது விவகாரம்: தமிழக, கர்நாடகா முதலமைச்சர்களுக்கு நிதின் கட்கரி கடிதம்
Cleaned:   மேகதாது விவகாரம் தமிழக கர்நாடகா முதலமைச்சர்களுக்கு நிதின் கட்கரி கடிதம்
Processed: மேகதாது விவகாரம் தமிழக கர்நாடகா முதலமைச்சர் நிதின் கட்கரி கடிதம்
--------------------------------------------------------------------------------

[Example 2]
Original:  பந்துவீச்சாளர்கள் ஐபிஎல் விளையாடலாமா? - எதிரெதிர் கருத்தில் தோனி-கும்பளே
Cleaned:   பந்துவீச்சாளர்கள் ஐபிஎல் விளையாடலாமா எதிரெதிர் கருத்தில் தோனி கும்பளே
Processed: பந்துவீச்சாளர் ஐபிஎல் விளையாடலாமா எதிரெதிர் கருத்தில் தோனி கும்பளே
--------------------------------------------------------------------------------

[Example 3]
Original:  கனமழை எச்சரிக்கை | நாளை பள்ளி, கல்லூரிகளுக்கு விடுமுறை.. எங்கெல்லாம் தெரியுமா?
Cleaned:   கனமழை எச்சரிக்கை நாளை பள்ளி கல்லூரிகளுக்கு விடுமுறை எங்கெல்லாம் தெரியுமா
Processed: கனமழை எச்சரிக்கை நாளை பள்ளி கல்லூரி விடுமுறை எங்கெல்லாம் தெரியுமா
---------------------------

In [86]:

processed_df['token_count'] = processed_df['processed_title'].apply(lambda x: len(x.split()))

print("\nToken Count Statistics:")
print(processed_df['token_count'].describe())




Token Count Statistics:
count    18448.000000
mean         8.062392
std          2.352002
min          1.000000
25%          7.000000
50%          8.000000
75%         10.000000
max         15.000000
Name: token_count, dtype: float64


In [87]:

final_df = processed_df[['category', 'processed_title']]  
final_df.to_csv('output/final_dataset.csv', index=False, encoding='utf-8')

print("\n✓ Final dataset exported to: output/final_dataset.csv")
print(f"  Shape: {final_df.shape}")


✓ Final dataset exported to: output/final_dataset.csv
  Shape: (18448, 2)


In [88]:
print(df['category'].value_counts())

category
இந்தியா                2192
உலகம்                  1894
குற்றம்                1627
தமிழ்நாடு              1591
சினிமா                 1456
டெக்                   1217
வணிகம்                 1208
சிறப்புக் களம்         1196
LIVE UPDATES            900
கிரிக்கெட்              895
ஹெல்த்                  849
சுற்றுச்சூழல்           639
விவசாயம்                614
T20                     459
டிரெண்டிங்              374
விளையாட்டு              356
கோலிவுட் செய்திகள்      204
ஆன்மீகம்                155
திரை விமர்சனம்           70
இலக்கியம்                59
ஓடிடி திரைப் பார்வை      52
கதைகள்                   49
லைஃப்ஸ்டைல்              44
மார்க்கெட்               42
கோயில்கள்                38
கொரோனா வைரஸ்             38
பட்ஜெட் 2025             28
தங்கம்                   28
கார்                     28
செஸ்                     21
பெண்கள்                  20
கல்வி                    18
மோட்டார்                 17
பாலிவுட் செய்திகள்       15
கால்பந்து                12
ஸ்மார்ட்ஃபோ